In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from data_with_pert import ModelNet40_pert,download,load_data
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
from matplotlib import pyplot as plt
import pickle
import warnings
import math
import time
from datetime import datetime
from collections import OrderedDict
from scipy.optimize import linear_sum_assignment
from torch import jit
%matplotlib inline
import os
from torch.utils.data import Dataset, TensorDataset
import importlib

In [2]:
train_losses = []
test_results_exp = []
test_accs = []

In [3]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 2 every 30 epochs"""
    lrt = lr * (0.5 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lrt

In [4]:
def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index)

In [5]:
device = torch.device("cuda:7")

In [6]:
M=10
batch_size= 8
data = ModelNet40_pert(num_points=1000, partition='train', gaussian_noise=False,
                       unseen=False, factor=4,n_pert=M)
data_test = ModelNet40_pert(num_points=1000, partition='test', gaussian_noise=False,
                       unseen=False, factor=4,n_pert=M)
train_loader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=True, drop_last=True)
rl_loader = DataLoader(data_test, batch_size=1, shuffle=True, drop_last=True)

/home/gwu/ConvexICP/data_with_pert.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [17]:
def train_eval_icp(model,device, train_loader,test_loader, optimizer,criterion, epoch, eval_mode='on'):
    global best
    model.train()
    final_cal = nn.Sigmoid()
    train_error_logs = []
    t1 = datetime.now()
    test_loss_log = []
    test_acc_log = []
    train_corrects = 0
    
    
    
    for batch_idx, (src, target, r_ab, t_ab, _,_,_,_,r_ab_pert,t_ab_pert,lambdas) in enumerate(train_loader):
        src = src.to(device)
        target = target.to(device)
        r_ab = r_ab.to(device)
        t_ab = t_ab.to(device)
        r_ab_pert = r_ab_pert.to(device)
        t_ab_pert = t_ab_pert.to(device)
        lambdas = lambdas.to(device)
        optimizer.zero_grad()
        dist, dist_pert, dist_pert_3,dist_pert_comb = model(src,target,r_ab,t_ab,r_ab_pert,t_ab_pert,lambdas)
        loss = criterion(dist,dist_pert, dist_pert_3, dist_pert_comb, target)
        loss.backward()
        optimizer.step()
        #pred_train = output_train.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        #train_corrects += pred_train.eq(target.view_as(pred_train)).sum().item()
        train_error_logs.append(loss.item())
        
        if (batch_idx+0) % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.2f}'.format(
                epoch, (batch_idx+0)* len(target), len(train_loader.dataset),
                100. * (batch_idx+0) / len(train_loader), loss,(datetime.now()-t1).total_seconds()))

        
    print('Train Epoch: {} Accuracy: {}/{} ({:.2f}%)\n'.format(
                epoch, train_corrects, len(train_loader.dataset),
                100. * train_corrects / len(train_loader.dataset)))
    '''
    model.eval()
    test_loss = 0
    correct_test = 0
    with torch.no_grad():
        for src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba in test_loader:
            src = src.to(device)
            target = target.to(device)
            rotation_ab = rotation_ab.to(device)
            translation_ab = translation_ab.to(device)
            rotation_ba = rotation_ba.to(device)
            translation_ba = translation_ba.to(device)
            output_test = model(src,target,rotation_ab,translation_ab)
            test_loss += criterion(output_test, target).item()*len(src)  # sum up batch loss
    test_loss /= len(test_loader.dataset)
    test_loss_log.append(test_loss)
    print('Test set: Average loss: {:.8f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct_test, len(test_loader.dataset),
            100. * correct_test / len(test_loader.dataset)))
    '''
    return train_error_logs,test_loss_log,test_acc_log

In [8]:
def cal_rt_loss(ra,ta,rb,tb):
    r_loss = F.mse_loss(ra,rb).item()
    t_loss = F.mse_loss(ta,tb).item()
    return r_loss, t_loss

In [9]:
class PointNet(nn.Module):
    def __init__(self, emb_dims=512):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv5 = nn.Conv1d(128, emb_dims, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(emb_dims)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        return x

In [37]:
class CPNet(nn.Module):
    def __init__(self, emb_dims=512,split = 10):
        super(CPNet, self).__init__()
        self.split = split
        self.base_net = PointNet()
        self.loss = SamplesLoss(loss="sinkhorn", p=2, blur=.05)

        
    def forward(self, x1, x2, R, T, R_pert, T_pert,lambdas):
        
        trans = torch.matmul(R, x1) + T.unsqueeze(2)
        emb1 = self.base_net(trans)
        emb2 = self.base_net(x2)
        
        dist = self.loss(emb1,emb2)#+self.loss(emb2,emb1)
        
        #constraint 1
        pert_out = torch.matmul(R_pert.permute(1,0,2,3), x1) + T_pert.permute(1,0,2).unsqueeze(3)
        pert_out = pert_out.reshape(-1, pert_out.shape[-2],pert_out.shape[-1])
        pert_emb1 = self.base_net(pert_out)
        pert_emb2 = tile(emb2,0,self.split)
        pert_dist = self.loss(pert_emb1,pert_emb2)/self.split #+ self.loss(pert_emb2,pert_emb1)/self.split
        pert_dist_1 = pert_dist.reshape(self.split, emb2.shape[0])
        
        #constraint 2
        
        lambdas_t = lambdas.unsqueeze(-1).expand(-1,-1,3)
        lambdas_r = lambdas_t.unsqueeze(-1).expand(-1,-1,-1,3)
        R_pert_3 = lambdas_r*R_pert+(1-lambdas_r)*tile(R,0,self.split).reshape(*R_pert.shape)
        T_pert_3 = lambdas_t*T_pert+(1-lambdas_t)*tile(T,0,self.split).reshape(*T_pert.shape)
        
        pert_out_3 = torch.matmul(R_pert_3.permute(1,0,2,3), x1) + T_pert_3.permute(1,0,2).unsqueeze(3)
        pert_out_3 = pert_out_3.reshape(-1, pert_out_3.shape[-2],pert_out_3.shape[-1])
        pert_emb1_3 = self.base_net(pert_out_3)
        pert_dist_3 = self.loss(pert_emb1_3,pert_emb2)/self.split #+ self.loss(pert_emb2,pert_emb1_3)/self.split
        pert_dist_3 = pert_dist_3.reshape(self.split, emb2.shape[0])
        
        pert_dist_comb = lambdas.T*pert_dist_1+(1-lambdas.T)*dist
        
        
        
        

        return dist,pert_dist_1.sum(0),pert_dist_3.sum(0),pert_dist_comb.sum(0)

In [52]:
def combine_loss(dist, pert_dist_1,pert_dist_3,pert_dist_comb, target, alpha = 1000, beta = 1000, eps = 0.005, eta = 0.005):
    loss = torch.mean(dist)
    
    diff = dist-pert_dist_1
    dummy = torch.zeros_like(diff)
    pert_loss = F.hinge_embedding_loss(diff,dummy,margin = eps)
    
    diff_2 = pert_dist_3-pert_dist_comb
    dummy_2 = torch.zeros_like(diff_2)
    pert_loss_2 = F.hinge_embedding_loss(diff_2,dummy_2,margin = eta)
    
    return loss+beta*pert_loss_2
#+alpha*pert_loss

In [53]:
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [54]:
model = CPNet(split=M).to(device)

In [55]:
criterion = combine_loss
optimizer = torch.optim.SGD(model.parameters(), lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
best=0

In [56]:
train_time= []
train_loss = []
train_losses.append(train_loss)
tests = []
test_acc = []
test_results_exp.append(tests)
test_accs.append(test_acc)
ratio = 0
for epoch in range(1,600):
    adjust_learning_rate(optimizer, epoch)
    t1 = datetime.now()
    train_error,test_error,test_acc_this = train_eval_icp(model,device, train_loader, test_loader, optimizer,criterion,epoch,'on')
    train_loss.extend(train_error)
    tests.extend(test_error)
    test_acc.extend(test_acc_this)
    train_time.append((datetime.now()-t1).total_seconds())
    print((datetime.now()-t1).total_seconds())

Train Epoch: 1 [0/9840 (0%)]	Loss: 296.131042	Time: 0.42
Train Epoch: 1 [160/9840 (2%)]	Loss: 32.612011	Time: 8.14
Train Epoch: 1 [320/9840 (3%)]	Loss: 37.148277	Time: 15.52
Train Epoch: 1 [480/9840 (5%)]	Loss: 54.193703	Time: 23.33
Train Epoch: 1 [640/9840 (7%)]	Loss: 23.446484	Time: 30.99
Train Epoch: 1 [800/9840 (8%)]	Loss: 27.590219	Time: 38.56
Train Epoch: 1 [960/9840 (10%)]	Loss: 15.318518	Time: 46.22
Train Epoch: 1 [1120/9840 (11%)]	Loss: 8.432614	Time: 53.68
Train Epoch: 1 [1280/9840 (13%)]	Loss: 6.981595	Time: 61.07
Train Epoch: 1 [1440/9840 (15%)]	Loss: 5.900253	Time: 68.25
Train Epoch: 1 [1600/9840 (16%)]	Loss: 5.772002	Time: 75.38
Train Epoch: 1 [1760/9840 (18%)]	Loss: 5.305846	Time: 82.50
Train Epoch: 1 [1920/9840 (20%)]	Loss: 5.371316	Time: 89.63
Train Epoch: 1 [2080/9840 (21%)]	Loss: 5.204899	Time: 96.75
Train Epoch: 1 [2240/9840 (23%)]	Loss: 5.317038	Time: 103.81
Train Epoch: 1 [2400/9840 (24%)]	Loss: 5.076724	Time: 110.74
Train Epoch: 1 [2560/9840 (26%)]	Loss: 5.075683

KeyboardInterrupt: 

In [46]:
torch.save(model.cpu(),"pert_model_new.mdl")

In [ ]:
for batch_idx, (src, target, r_ab, t_ab, _,_,_,_,r_ab_pert,t_ab_pert,r_ab_pert_2,t_ab_pert_2,lambdas) in enumerate(train_loader):
    print(lambdas.shape,r_ab_pert.shape)
    src = src.to(device)
    target = target.to(device)
    r_ab = r_ab.to(device)
    t_ab = t_ab.to(device)
    r_ab_pert = r_ab_pert.to(device)
    t_ab_pert = t_ab_pert.to(device)
    r_ab_pert_2 = r_ab_pert_2.to(device)
    t_ab_pert_2 = t_ab_pert_2.to(device)
    lambdas = lambdas.to(device)
    
    break

In [ ]:
torch.cuda.empty_cache()

In [ ]:
plt.plot(train_loss)

In [ ]:
plt.plot(train_loss[:30])